Automating Internal Transaction Removal

**Simplified Problem:** We have financial figures (Revenue, Expense) from two related entities, A and B. A sold something to B internally for 50,000. When combining their figures, this internal transaction needs to be removed to get a true picture of the group's performance with the outside world.

**Goal:** Demonstrate a simple, automated way using Python (pandas) to perform this removal, highlighting the logic.

**Scenario:**
*   Entity A: Has external revenue/expense, plus 50,000 revenue from selling to B.
*   Entity B: Has external revenue/expense, plus 50,000 expense from buying from A.

## 1. Input Data Setup

In [11]:
import pandas as pd

pd.options.display.float_format = '{:,.0f}'.format

# Data for Entity A
data_a = {
    # Clear labels distinguishing internal vs external
    'Account': ['Revenue_External', 'Revenue_Internal_to_B', 'Expense_External'],
    'Amount': [550000, 50000, -400000] # Revenue +, Expense -
}
entity_a_df = pd.DataFrame(data_a).set_index('Account')

# Data for Entity B
data_b = {
    'Account': ['Revenue_External', 'Expense_External', 'Expense_Internal_from_A'],
    'Amount': [400000, -150000, -50000] # Revenue +, Expense -
}
entity_b_df = pd.DataFrame(data_b).set_index('Account')

print("--- Input Data A ---")
print(entity_a_df)
print("\n--- Input Data B ---")
print(entity_b_df)

--- Input Data A ---
                       Amount
Account                      
Revenue_External       550000
Revenue_Internal_to_B   50000
Expense_External      -400000

--- Input Data B ---
                         Amount
Account                        
Revenue_External         400000
Expense_External        -150000
Expense_Internal_from_A  -50000


## 2. Manual / Naive Combination (The Problem)

Simply adding the figures together without removing the internal transaction gives incorrect totals.

In [23]:
# Combine dataframes based on Account names
combined_df = pd.concat([
    entity_a_df.rename(columns={'Amount': 'A'}),
    entity_b_df.rename(columns={'Amount': 'B'})
], axis=1).fillna(0)

print(combined_df)

# Calculate the simple sum
combined_df['Naive_Sum'] = combined_df['A'] + combined_df['B']

print("--- Combined Figures (Naive Sum) ---")
print(combined_df)

# Calculate Naive Profit
def calculate_profit(df, column_name):
    revenue = df.loc[df.index.str.contains('Revenue'), column_name].sum()
    expense = df.loc[df.index.str.contains('Expense'), column_name].sum()
    profit = revenue + expense # expense is negative
    return revenue, expense, profit

naive_revenue, naive_expense, naive_profit = calculate_profit(combined_df, 'Naive_Sum')

print("\n--- Naive Calculation ---")
print(f"Naive Total Revenue: {naive_revenue:,.0f}")
print(f"Naive Total Expense: {naive_expense:,.0f}")
print(f"Naive Profit:        {naive_profit:,.0f}")
print("Error: Internal 50,000 is double-counted (once as Revenue, once as Expense).")

                               A        B
Account                                  
Revenue_External         550,000  400,000
Revenue_Internal_to_B     50,000        0
Expense_External        -400,000 -150,000
Expense_Internal_from_A        0  -50,000
--- Combined Figures (Naive Sum) ---
                               A        B  Naive_Sum
Account                                             
Revenue_External         550,000  400,000    950,000
Revenue_Internal_to_B     50,000        0     50,000
Expense_External        -400,000 -150,000   -550,000
Expense_Internal_from_A        0  -50,000    -50,000

--- Naive Calculation ---
Naive Total Revenue: 1,000,000
Naive Total Expense: -600,000
Naive Profit:        400,000
Error: Internal 50,000 is double-counted (once as Revenue, once as Expense).


## 3. Automated Elimination Logic

We define the internal transaction(s) and create rules for generating adjustments. This mimics a rules engine approach.

In [15]:
# 1. Define the known internal transactions
# In a real system, this would be from a database or input file
internal_transactions_data = [
    {'ID': 'TXN123', 'Type': 'Sale', 'From': 'A', 'To': 'B', 'Amount': 50000}
    # Add more transactions here if needed
]

# 2. Define the rules for creating adjustments based on transaction type
# This is the core logic engine.
def get_adjustment_rules(transaction):
    txn_type = transaction['Type']
    from_entity = transaction['From']
    to_entity = transaction['To']
    amount = transaction['Amount']
    adjustments = []

    if txn_type == 'Sale':
        # Rule 1: Reduce the internal revenue of the seller ('From')
        revenue_account_label = f"Revenue_Internal_to_{to_entity}"
        adjustments.append({'Account': revenue_account_label, 'Adjustment': -amount, 'Reason': f"Remove {txn_type} Revenue"})

        # Rule 2: Cancel the internal expense of the buyer ('To')
        # (Expense is negative, so adding the amount cancels it)
        expense_account_label = f"Expense_Internal_from_{from_entity}"
        adjustments.append({'Account': expense_account_label, 'Adjustment': amount, 'Reason': f"Remove {txn_type} Expense"})

    # --- Add rules for other transaction types ('Loan', 'Service', etc.) here --- 

    return adjustments

# 3. Generate all adjustments by applying rules to transactions
all_adjustments_list = []
for txn in internal_transactions_data:
    txn_adjustments = get_adjustment_rules(txn)
    all_adjustments_list.extend(txn_adjustments)

# 4. Aggregate adjustments by account (in case multiple adjustments affect the same account)
adjustments_df = pd.DataFrame(all_adjustments_list)
aggregated_adjustments = adjustments_df.groupby('Account')['Adjustment'].sum().reset_index()
aggregated_adjustments = aggregated_adjustments.set_index('Account')

print("--- Automated Adjustments Generated ---")
print(aggregated_adjustments)

# Sanity check: Revenue/Expense eliminations should sum to zero
print(f"\nSum of Adjustments: {aggregated_adjustments['Adjustment'].sum():,.0f} (Should be 0 for Rev/Exp elims)")

--- Automated Adjustments Generated ---
                         Adjustment
Account                            
Expense_Internal_from_A       50000
Revenue_Internal_to_B        -50000

Sum of Adjustments: 0 (Should be 0 for Rev/Exp elims)


## 4. Applying Adjustments for Final Correct Result

In [17]:
# Join the generated adjustments to the combined figures
final_df = combined_df.join(aggregated_adjustments, how='left').fillna(0)

# Apply the adjustment: Final Amount = Naive Sum + Adjustment
final_df['Final_Amount'] = final_df['Naive_Sum'] + final_df['Adjustment']

print("--- Combined Figures After Adjustments ---")
print(final_df[['Naive_Sum', 'Adjustment', 'Final_Amount']]) # Show the calculation

# Calculate Final Correct Profit
final_revenue, final_expense, final_profit = calculate_profit(final_df, 'Final_Amount')

print("\n--- Final Calculation ---")
print(f"Final Total Revenue: {final_revenue:,.0f}")
print(f"Final Total Expense: {final_expense:,.0f}")
print(f"Final Profit:        {final_profit:,.0f}")

# Verify internal accounts are now zero
internal_accounts_check = [
    'Revenue_Internal_to_B', 'Expense_Internal_from_A'
]
print("\nCheck: Final Amounts for purely internal accounts:")
print(final_df.loc[final_df.index.isin(internal_accounts_check), ['Final_Amount']])

--- Combined Figures After Adjustments ---
                         Naive_Sum  Adjustment  Final_Amount
Account                                                     
Revenue_External           950,000           0       950,000
Revenue_Internal_to_B       50,000     -50,000             0
Expense_External          -550,000           0      -550,000
Expense_Internal_from_A    -50,000      50,000             0

--- Final Calculation ---
Final Total Revenue: 950,000
Final Total Expense: -550,000
Final Profit:        400,000

Check: Final Amounts for purely internal accounts:
                         Final_Amount
Account                              
Revenue_Internal_to_B               0
Expense_Internal_from_A             0


## 5. SWE Perspective & Benefits

*   **Modularity:** The logic for generating adjustments (`get_adjustment_rules` function) is separate from the data loading and final calculation. This makes the code easier to manage and test.
*   **Scalability:** Adding more entities just means adding their DataFrames to the `pd.concat` step. Adding new *types* of internal transactions means adding new `if` blocks or rules to the `get_adjustment_rules` function.
*   **Data Driven:** The process is driven by the `internal_transactions_data` list. In a real system, this data would likely come from a database, making the process highly automated.
*   **Consistency:** The same rules are applied every time, reducing human error compared to manual spreadsheet adjustments.
*   **Readability:** Using clear account labels and rule descriptions helps understand the process.